In [ ]:
from helper import *

In [ ]:
pd.set_option('display.max_rows', None)

# load preds
preds_2025 = pd.read_csv('./data/clean/predictions_2025.csv')

pos = 'QB'
preds_2025.query('pos == @pos').sort_values('ppg_pred', ascending=False).head(36)

teams_2024 = pd.read_csv('./data/pff/team/2024.csv')
teams_2024['blocking grade'] = (teams_2024['Pass Blocking Grade'] + teams_2024['Run Blocking Grade']).div(2)
teams_2024 = teams_2024[['Tm', 'Points For', 'Points Against', 'blocking grade', 'Pass Blocking Grade', 'Run Blocking Grade']].copy().sort_values('blocking grade', ascending=False).reset_index(drop=True)
teams_2024